# Dataset for Aspect-Based Sentiment Analysis and Topic Modeling on Borobudur Temple and Prambanan Temple by Dian Arianto
https://github.com/dian9395/dataset-analisis-sentimen-berbasis-aspek-dan-pemodelan-topik

In [3]:
import pandas as pd
import demoji
import unicodedata as uni

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [4]:
df = pd.read_excel('dataset/reviews_borobudur_prambanan_TripAdvisor_GMaps_all_tesis.xlsx')
df_clean = df.dropna()
# print("\nDrop rows with any NaN values:")
df_clean

,id,lokasi,text,daya_tarik,amenitas,aksesibilitas,citra,harga,sdm
0,1.0,Candi Borobudur,peninggalan sejarah yang sudah berumur 1200 ta...,1,-,-,1,0,0
1,2.0,Candi Borobudur,Pertama kali bepergian selama masa pandemi. Ca...,-,1,-,1,-,1
2,3.0,Candi Borobudur,"Candi Borobudur di Magelang, Yogyakarta adalah...",1,-,-,1,-,-
3,4.0,Candi Borobudur,"Baru pertama kali kesini, pas sih kalau tempat...",1,-,-,-,-1,-
4,5.0,Candi Borobudur,"candi borobudur, tempat wisata ini sudah terke...",1,1,-,1,-,-
...,...,...,...,...,...,...,...,...,...
6738,6739.0,Candi Prambanan,The place is great and probably everything you...,1,-,-,-,-1,-
6739,6740.0,Candi Prambanan,Prambanan or Rara Jonggrang (Javanese: ꦫꦫꦗꦺꦴꦁꦒ...,0,-,-,-,-,-
6740,6741.0,Candi Prambanan,This temple is not worth the price. There’s no...,-1,-,-,-1,-1,-1
6741,6742.0,Candi Prambanan,Prambanan temple is the largest hindu temple o...,1,-,-,-,-,-


## Preprocessing
Emoji processing; case folding; menghapus username, angka, dan tanda baca; koreksi ejaan dan singkatan serta menghapus whitespace; penghapusan stopwords, dan stemming.

In [ ]:
def preprocess(text):
    # Text normalization
    contractions = {
        "isn't": "is not",
        "aren't": "are not",
        "wasn't": "was not",
        "weren't": "were not",
        "haven't": "have not",
        "hasn't": "has not",
        "hadn't": "had not",
        "won't": "will not",
        "wouldn't": "would not",
        "don't": "do not",
        "doesn't": "does not",
        "didn't": "did not",
        "can't": "can not",
        "couldn't": "could not",
        "shouldn't": "should not",
        "mightn't": "might not",
        "mustn't": "must not",
        "I'm": "I am",
        "You're": "you are",
        "I've": "I have",
        "UI": "user interface",
        "UX": "user experience",
        "u": "you",
    }
    pattern = re.compile(r'\b(' + '|'.join(contractions.keys()) + r')\b')
    expanded_text = pattern.sub(lambda match: contractions[match.group(0)], text)

    normalized_text = uni.normalize('NFKD', expanded_text)
    normalized_text = ''.join([c for c in normalized_text if not uni.combining(c)])

    # emoji encoding
    emojis = demoji.findall(text)

    for emoji in emojis:
        text = text.replace(emoji, " " + emojis[emoji].split(":")[0])

    # text preprocessing
    teks = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    teks = teks.lower()
    stop_words = set(stopwords.words('english'))
    stop_words.update(['also', 'app', 'apps', 'application', 'applications', 'good'])
    stop_words.remove('not')
    tokens = word_tokenize(teks)
    filtered_tokens = [word for word in tokens if word.isalnum() and not any(char.isdigit() for char in word) and word not in stop_words]
    lemmatizer = nltk.WordNetLemmatizer()
    lemma = [lemmatizer.lemmatize(word) for word in filtered_tokens]
    clean_reviews = ' '.join(lemma)

    return clean_reviews